In [ ]:
# !pip install pandas scikit_learn surprise
# !pip install openpyxl
# !pip install transformers
# !pip install matplotlib seaborn
# !pip install wordcloud
# !pip install tensorflow
# !pip install tf-keras
# !pip install nltk
# !pip install pyarrow
# !pip install ollama

In [ ]:

import pandas as pd
from transformers import pipeline
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import plotly.graph_objects as go

import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

In [ ]:
# df = pd.read_excel('/Users/maralsheikhzadeh/Documents/Codes/Server-Connect/etrusted_reviews_2024-07-17_09-31.xlsx')
df = pd.read_parquet('Exports/sentiment_analysis.parquet')



In [ ]:
## Selecting the needed columns
df = df[['Bewertungs-ID',
 'Erstellt-Datum',
 'E-Mail',
 'Bewertung',
 'Bewertungstitel',
 'Bewertungstext',
 'Beantwortet-Datum',
 'Antworttext',
 'Referenz',
 'Vorlage',
 'Produkt-ID',
 'Produkt-SKU',
 'Produktname',
 'Produkt-URL',
 'Produktbild-URL',
 'sentiment']]




## Counting Positive, Neutral and Negativ reviews for each Product

In [ ]:


# # Custom aggregation function to count each sentiment separately
# def count_sentiments(sentiments):
#     sentiment_counts = sentiments.value_counts()
#     positive = sentiment_counts.get('POSITIVE', 0)
#     neutral = sentiment_counts.get('NEUTRAL', 0)
#     negative = sentiment_counts.get('NEGATIVE', 0)
#     return pd.Series({'POSITIVE': positive, 'NEUTRAL': neutral, 'NEGATIVE': negative})

# # Group by 'Produkt-SKU' and apply the custom aggregation function
# average_review = df.groupby('Produkt-SKU').agg(
#     Bewertungstext=('Bewertungstext', ' *** '.join),
#     Positive_Count=('sentiment_mapped', lambda x: count_sentiments(x)['POSITIVE']),
#     Neutral_Count=('sentiment_mapped', lambda x: count_sentiments(x)['NEUTRAL']),
#     Negative_Count=('sentiment_mapped', lambda x: count_sentiments(x)['NEGATIVE'])
# ).reset_index()




# average_review.to_csv('average_reviews.csv',index=False)

## Applying Sentiment analysis model (Takes a while to process)

In [ ]:
# ## Applying Sentiment analysis model to the data
# sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
# df['sentiment'] = df['cleaned_reviews'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

In [ ]:
## Renaming/Mapping sentiment results to POSITIVE,NEUTRAL and NEGATIVE values
label_mapping = {
    '1 star': 'NEGATIVE',
    '2 stars': 'NEGATIVE',
    '3 stars': 'NEUTRAL',
    '4 stars': 'POSITIVE',
    '5 stars': 'POSITIVE'
}

df['sentiment_mapped'] = df['sentiment'].map(label_mapping)

## Saving the df to Parquet format
# df.to_parquet('sentiment_analysis.parquet',index=False)

## Formatting the Date column as date type
df['Erstellt-Datum'] = pd.to_datetime(df['Erstellt-Datum'], format='%d.%m.%y, %H:%M')
df['Erstellt-Datum'] = df['Erstellt-Datum'].dt.strftime('%Y-%m-%d')
# df.set_index('Erstellt-Datum',inplace=True)


## Transforming data to a Pivot Table

In [ ]:
new_df = df[['Erstellt-Datum','Produkt-SKU', 'Bewertungstext','sentiment_mapped']]

def aggregate_reviews(texts):
    unique_texts = list(set(texts))
    return ' *** '.join(unique_texts)

# Create the pivot table
pivot_df = new_df.pivot_table(index=['Erstellt-Datum', 'Produkt-SKU'], 
                          columns='sentiment_mapped', 
                          values='Bewertungstext', 
                          aggfunc=aggregate_reviews, 
                          fill_value='')

# Reset the index to flatten the DataFrame
pivot_df = pivot_df.reset_index()

# Rename columns for clarity
pivot_df.columns.name = None
pivot_df = pivot_df[pivot_df['Produkt-SKU'].str.endswith('P')==False]
pivot_df.to_excel('Exports/pivot_reviews.xlsx',engine='openpyxl')

## Extracting Most frequent Phrases from the reviews

In [ ]:
import spacy

# Load the German language model
nlp = spacy.load('de_core_news_sm')

def extract_noun_verb_phrases(text):
    doc = nlp(text)
    noun_phrases = [chunk.text for chunk in doc.noun_chunks]
    verb_groups = [token.text for token in doc if token.pos_ == 'VERB']
    return noun_phrases + verb_groups  # Concatenate noun_phrases and verb_groups into a single list

# Apply the function to each row in the 'Bewertungstext' column
df['Noun_Phrases'] = df['Bewertungstext'].str.lower()
df['Noun_Phrases'] = df['Noun_Phrases'].apply(extract_noun_verb_phrases)

# Handling NaN values
df['Noun_Phrases'] = df['Noun_Phrases'].fillna('0')


## Finding Most Frequent Phrases in Reviews

In [ ]:
# Step 1: Split the text into words (tokenization)
texts = df['Noun_Phrases']


# Step 2: Filter out empty lists
new = texts[texts.apply(lambda x: isinstance(x, list) and len(x) > 0)].values.tolist()


# Step 3: Flatten the list of lists
corpus = [word for sublist in new for word in sublist]


# Step 4: Count the words
counter = Counter(corpus)
most = counter.most_common()


# Step 5: Prepare the data for plotting
x, y = [], []
german_stop_words = [
    "aber", "als", "am", "an", "auf", "aus", "bei", "bin", "bis", "da", "durch", 
    "ein", "eine", "einer", "eines", "einem", "einen", "es", "für", "gegen", 
    "habe", "haben", "hat", "hatte", "hier", "ich", "in", "ist", "ja", "jedoch", 
    "kann", "könnte", "machen", "mein", "meine", "mit", "nach", "nicht", "nur", 
    "oder", "sein", "seine", "sind", "so", "über", "um", "und", "unser", "unsere", 
    "von", "vor", "was", "weil", "wie", "will", "wir", "wird", "wo", "zu", "zum", 
    "zur",'-','##e','.','!','Ich','war','sehr','alles','allem','der','die','das'
]

for word, count in most[:300]:
    if word not in german_stop_words:
        ## We only want two or longer phrases that are descriptive (hence with no articles)
        if (len(word.split()) > 1) and (word.split()[0] not in ['der', 'die', 'das']):
            print(word)
            x.append(word)
            y.append(count)

# Step 6: Create the bar plot
fig = go.Figure(go.Bar(
    x=y,
    y=x,
    orientation='h',
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1
        )
    ),
    name='Most common Word'
))

fig.update_layout(
    title={
        'text': "Most Common Words",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Courier New, monospace",
        size=11,
        color="RebeccaPurple"
    )
)

fig.show()


## Computing Average Rating and number of rating for each product

In [ ]:
rating_sorted = df.groupby(['Produkt-SKU', 'Produktname']).agg(
    Durchschnitt_Bewertungen=('Bewertung', 'mean'),
    Anzahl_Bewertungen=('Bewertung', 'count')
).sort_values('Anzahl_Bewertungen', ascending=False).reset_index()

rating_sorted.to_csv('Exports/all_product_ratings.csv')

### Top and Flop rated Products based on both the Average rating and the number of ratings

In [ ]:
# rating_sorted = rating_sorted[rating_sorted['sentiment_mapped']]
rating_sorted['rating'] = rating_sorted['Durchschnitt_Bewertungen'] * rating_sorted['Anzahl_Bewertungen']
top_rated = rating_sorted.sort_values('rating',ascending=False)[:100]
# top_rated = top_rated.rename(columns={'Produkt-SKU':'NUMMER','Bewertung':'Bewertung (Durschnitt)','sentiment_mapped':'Anzahl die Bewertungen'})
top_rated = top_rated[['Produkt-SKU','Produktname','Durchschnitt_Bewertungen','Anzahl_Bewertungen']]
top_rated.to_csv('Exports/top_rated_products.csv')

In [ ]:
# rating_sorted = rating_sorted[rating_sorted['sentiment_mapped']]
# rating_sorted['rating'] = rating_sorted['Bewertung'] * rating_sorted['sentiment_mapped']
flop_rated = rating_sorted.sort_values(['Durchschnitt_Bewertungen','Anzahl_Bewertungen'],ascending=[True,True])
flop_rated = flop_rated[(flop_rated['Anzahl_Bewertungen']>=2)& (flop_rated['Durchschnitt_Bewertungen'] <= 3)][:100]
# flop_rated = flop_rated.rename(columns={'Produkt-SKU':'NUMMER','Bewertung':'Bewertung (Durschnitt)','sentiment_mapped':'Anzahl die Bewertungen'})
flop_rated = flop_rated[['Produkt-SKU','Produktname','Durchschnitt_Bewertungen','Anzahl_Bewertungen']]
flop_rated.to_csv('Exports/flop_rated_products.csv')

In [ ]:
sentiment_summary = df.groupby('Produkt-SKU')['sentiment_mapped'].value_counts().unstack().fillna(0)
sentiment_summary.sort_values(by='POSITIVE',ascending=False)

In [ ]:
# sentiment_summary.plot(kind='bar', stacked=True)
# plt.title('Sentiment Distribution by Product')
# plt.xlabel('Product ID')
# plt.ylabel('Number of Reviews')
# plt.show()

## Creating Wordcloud

In [ ]:
# Define a function to clean text and remove stopwords
def clean_text(text):
    if isinstance(text, str):  # Ensure the text is a string
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word.lower() not in german_stop_words]
        return ' '.join(tokens)
    return ''  # Return an empty string if not a valid string

# Clean the reviews to remove stopwords
positive_reviews = ' '.join(df[df['sentiment_mapped'] == 'POSITIVE']['Bewertungstext'].dropna().astype(str))
negative_reviews = ' '.join(df[df['sentiment_mapped'] == 'NEGATIVE']['Bewertungstext'].dropna().astype(str))

# Clean the concatenated reviews
cleaned_positive_reviews = clean_text(positive_reviews)
cleaned_negative_reviews = clean_text(negative_reviews)

# Generate word clouds with cleaned text
wordcloud_pos = WordCloud(width=800, height=400, background_color='white').generate(cleaned_positive_reviews)
wordcloud_neg = WordCloud(width=800, height=400, background_color='white').generate(cleaned_negative_reviews)

# Plotting
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis('off')
plt.title('Positive Reviews Word Cloud')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis('off')
plt.title('Negative Reviews Word Cloud')

plt.tight_layout()  # Ensures plots are neatly arranged
plt.show()
